In [2]:
import pandas as pd
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [36]:
#input data
def input_data(folder):
    cv = pd.read_csv(folder+'/' + folder + '_photos_calib_result.csv')
    cv['pupil'] = pd.to_numeric(cv['pupil'], errors='coerce')
    cv['ratio'] = pd.to_numeric(cv['ratio'], errors='coerce')
    cv['x'] = pd.to_numeric(cv['x'], errors='coerce')
    cv['y'] = pd.to_numeric(cv['y'], errors='coerce')
    cv['ts'] = pd.to_numeric(cv['ts'], errors='coerce')
    cv['ts'] = np.around(cv['ts'], decimals=0)
    #seperate by steps
    cv = cv[cv['pupil'] > 0] #remove zeros
    
    s30_cv = cv[cv['step'] == 30]
    s31_cv = cv[cv['step'] == 31]
    s32_cv = cv[cv['step'] == 32]
    
    return cv, s30_cv, s31_cv, s32_cv

def input_eve_data(folder, step, step_num):
    left_pupil_db = pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['left_p']['data'])) #in mm
    left_pupil_valid =  pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['left_p']['validity']))
    right_pupil_db = pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['right_p']['data']))
    right_pupil_valid =  pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['right_p']['validity']))
    
#     px_2_mm = pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['pixels_per_millimeter']))
    gaze_right = pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['right_o']['data']))
    gaze_left = pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['left_o']['data']))
    gaze_left = gaze_left.rename(columns={0: 'x', 1: 'y', 2: 'z'})
    gaze_right = gaze_right.rename(columns={0: 'x', 1: 'y', 2: 'z'})
    
    mm_per_px = pd.DataFrame(np.array(h5.File(folder+'/' + step + '_webcam_c.h5')['millimeters_per_pixel']))
    mm_per_px = mm_per_px[0][0]
    
    #normalise input data
    #eve db
    #convert mm into pixels
    left_pupil_db /= mm_per_px
    right_pupil_db /= mm_per_px
    
    #insert the timestamp information for the eve data
    ts_eve = attach_ts(step_num, folder)
    left_pupil_db['ts'] = ts_eve
    
    # integrate into one dataframe with the coordinate
    left_pupil_db['x'] = gaze_left['x']
    left_pupil_db['y'] = gaze_left['y']
    left_pupil_db['validity'] = left_pupil_valid

    right_pupil_db['ts'] = ts_eve
    right_pupil_db['x'] = gaze_right['x']
    right_pupil_db['y'] = gaze_right['y']
    right_pupil_db['validity'] = right_pupil_valid

    left_pupil_db = left_pupil_db.rename(columns={0: 'pupil'})
    right_pupil_db = right_pupil_db.rename(columns={0: 'pupil'})
    left_pupil_db = left_pupil_db[left_pupil_db['pupil'] != 0]
    right_pupil_db = right_pupil_db[right_pupil_db['pupil'] != 0]
    
    #almost all the data is valid
    return left_pupil_db[left_pupil_db['validity'] == True], right_pupil_db[right_pupil_db['validity'] == True], ts_eve

def attach_ts(stepnum, folder):
    f = open(folder+'/step_' + stepnum +'_webcam_c.timestamps.txt','r')
    ts = []
    for line in f.readlines():
        ts.append(float(line[:-1]))

    start = ts[0]
    for i in range(len(ts)):
        ts[i] -= start
        ts[i] /= 1000000 #from ms to s
        ts[i] = np.around(ts[i], decimals=0)
    return ts


In [37]:
from statsmodels.stats.stattools import medcouple
import math

def get_quartiles(arr):
    arr = [i for i in arr if not np.isnan(i)]
    arr = np.sort(arr)
    mid = int(len(arr)/2)
    if(len(arr)%2 == 0):
        Q1 = np.median(arr[:mid])
        Q3 = np.median(arr[mid:])
    else:
        Q1 = np.median(arr[:mid])
        Q3 = np.median(arr[mid:])
    return Q1,Q3

def adjust_boxplot(values, param, bias=1.5):
    mc = np.around(medcouple(values), decimals=1)
    
    if mc == 0: #tukeys method
        q1,q3 = get_quartiles(values)
        iqr = q3 - q1
        lowerLimit = np.around(q1 - bias*iqr, decimals=2)
        upperLimit = np.around(q3 + bias*iqr, decimals=2)
    else:
        q1,q3 = get_quartiles(values)
        iqr = q3 - q1
        if mc > 0:
            lowerLimit = np.around(q1 - bias*math.exp(-3.5*mc)*iqr, decimals=2)
            upperLimit = np.around(q3 + bias*math.exp(4*mc)*iqr, decimals=2)
        else:
            lowerLimit = np.around(q1 - bias*math.exp(-4*mc)*iqr, decimals=2)
            upperLimit = np.around(q3 + bias*math.exp(3.5*mc)*iqr, decimals=2)
        
    result = []
    for v in values:
        if v < lowerLimit or v > upperLimit: 
            result.append(0)
        else:
            result.append(v)
    
    remove = np.around(1-(len([i for i in result if i != 0])/len(values)), decimals=2)

    return result

def remove_outliers(df, param, bias=1.5): 
    ad_value = adjust_boxplot(list(df[param]), param, bias)
    df[param] = ad_value
    df = df[df[param] != 0]
    
    return df


In [38]:
#merge function for cv db
def merge_both_side(ts, df_left, df_right):
    data = []
    
    for t in ts:
        left_series = df_left[df_left['ts'] == t]
        right_series = df_right[df_right['ts'] == t]
        
        if len(left_series) != 0:
            left = list(left_series['pupil'])[0]
            y_left = list(left_series['y'])[0]
            x_left = list(left_series['x'])[0]
        else:
            left, y_left, x_left = np.nan, np.nan, np.nan
        
        if len(right_series) != 0:
            right = list(right_series['pupil'])[0]
            y_right = list(right_series['y'])[0]
            x_right = list(right_series['x'])[0]
        else:
            right, y_right, x_right = np.nan, np.nan, np.nan
        
        if not np.isnan(left) and not np.isnan(right):
            data.append([t, np.mean([left, right])])
        elif not np.isnan(right):
            data.append([t, right])
        elif not np.isnan(left):
            data.append([t, left])
        else:
            data.append([t, np.nan])
        
    df = pd.DataFrame(data, columns=['ts', 'pupil'])
    df['pcps'] = df['pupil'].pct_change()
    df = df[df['pupil'] != np.nan]
    return df


In [39]:
### normalise into 1sec
def nor_1000(df, sec=1):
    video_length = np.around(list(df['ts'])[-1]/1000, decimals=0)

    times = np.arange(0, video_length*1000, (1000*sec)) #interval
    start = 0
    data = []
    for t in times[1:]:
        rows = df[ (df['ts'] >= start) & (df['ts'] <= t)]
        values = sorted([i for i in rows['pupil'] if i != 0])
        data.append([start, np.around(np.nanmean(values), decimals=3)])
        start = t
        
    result = pd.DataFrame(data, columns=['ts', 'pupil'])   
    result['pcps'] = result['pupil'].pct_change()
    return result 

def preprocess(step_cv_df, folder, step, step_num):
    ts_cv = []
    for t in list(step_cv_df['ts']): #the whole ts list before removing any values
        if t not in ts_cv:
            ts_cv.append(t)
    
    #input eve data
    left_pupil_eve, right_pupil_eve, ts_eve = input_eve_data(folder, step, step_num)
    
    #clean cv data
    before_size = len(list(step_cv_df['pupil']))
    step_cv_df = remove_outliers(step_cv_df, 'pupil')
    step_cv_df = remove_outliers(step_cv_df, 'y', 3)
    step_cv_df = remove_outliers(step_cv_df, 'x', 3)
    
    print('** after clean cv data, remove ratio', 1-(len(list(step_cv_df['pupil']))/before_size), '\n')

    changes_cv_left = step_cv_df[step_cv_df['side'] == 'L'] #select individual side
    changes_cv_right = step_cv_df[step_cv_df['side'] == 'R']
    
    #clean eve data
    left_pupil_eve = remove_outliers(left_pupil_eve, 'pupil')
    right_pupil_eve = remove_outliers(right_pupil_eve, 'pupil')
    
    cv = merge_both_side(ts_cv, changes_cv_left, changes_cv_right)
    eve = merge_both_side(ts_eve, left_pupil_eve, right_pupil_eve) 
    
    video_length = np.around(list(eve['ts'])[-1]/1000, decimals=0)
    freq_eve = np.around(len(eve['pupil'])/video_length, decimals=0)
    freq_cv = np.around(len(cv['pupil'])/video_length, decimals=0)
    print('eve freq = ', freq_eve, ' | cv freq = ', freq_cv)
    return cv, eve

In [40]:
from scipy.stats import pearsonr      

def windowed_corr(df, window=3, overlap=0):
    #overall pearson correlation    
    df = df.dropna(subset=['pupil'])
        
    r, p = pearsonr(list(df['ts']), list(df['pupil']))
    print('overall pearson correlation (r, p):', np.around(r, decimals=3), np.around(p, decimals=3))
    return r,p
#     #windowed pearson correlation
#     print('\t-- with window --')
#     length = list(df['ts'])[-1]

#     idx = np.arange(0, length, (window-overlap)*1000)
#     positive, negative, total = 0, 0, 0
    
#     start = 0
#     for t in idx[1:]:
#         rows = df[(df['ts'] >= start) & (df['ts'] < t)]

#         if len(rows) >= 2: 
#             corr = pearsonr(list(rows['ts']), list(rows['pupil']))
#         else:
#             corr = [0,0]
#         print('\t\tcorrelation (r, p) at window ', window, ':', np.around(corr, decimals=3))
#         total += 1
        
#         if corr[0] > 0.5:
#             positive += 1
#         if corr[0] < -0.5:
#             negative += 1
#         start = t
    
#     print('position cor, negative cor, total:', positive, negative, total)


In [41]:
from scipy import fft

def run(step_num, step_cv_df, folder):
    step_num = str(step_num)
    step = 'step_' + step_num
    save_file = folder+'/'+step
    suffix = 'video' 
    cv, eve = preprocess(step_cv_df, folder, step, step_num)
    print(f'\n**{step}')
    
    for i in range(6):
        try:
            print(f'\t-- {i+1} SEC Timeframe --')
            cv_new = nor_1000(cv, i+1)
            eve_new = nor_1000(eve, i+1)
            print('cv length', len(cv_new), 'eve length', len(eve_new))
            print('[cv]')
            rcv, pcv = windowed_corr(cv_new)
            print('[eve]')
            re, pe = windowed_corr(eve_new)
            print(f'-- diff: {np.around(rcv-re, decimals=3)}, {np.around(pcv-pe, decimals=3)}\n')
            
        except:
            continue

In [42]:
cv_36, s30_cv36, s31_cv36, s32_cv36 = input_data('train36')
run('30', s30_cv36, 'train36')
run('31', s31_cv36, 'train36')
run('32', s32_cv36, 'train36')

C:\Users\kaixi\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


** after clean cv data, remove ratio 0.04397705544933084 

eve freq =  30.0  | cv freq =  27.0

**step_30
	-- 1 SEC Timeframe --
cv length 9 eve length 9
[cv]
overall pearson correlation (r, p): 0.28 0.465
[eve]
overall pearson correlation (r, p): 0.147 0.706
-- diff: 0.133, -0.241

	-- 2 SEC Timeframe --
cv length 4 eve length 4
[cv]
overall pearson correlation (r, p): 0.39 0.61
[eve]
overall pearson correlation (r, p): 0.041 0.959
-- diff: 0.349, -0.349

	-- 3 SEC Timeframe --
cv length 3 eve length 3
[cv]
overall pearson correlation (r, p): 0.99 0.089
[eve]
overall pearson correlation (r, p): 0.356 0.768
-- diff: 0.634, -0.679

	-- 4 SEC Timeframe --
cv length 2 eve length 2
[cv]
overall pearson correlation (r, p): 1.0 1.0
[eve]
overall pearson correlation (r, p): 1.0 1.0
-- diff: 0.0, 0.0

	-- 5 SEC Timeframe --
cv length 1 eve length 1
[cv]
	-- 6 SEC Timeframe --
cv length 1 eve length 1
[cv]
** after clean cv data, remove ratio 0.11759082217973227 

eve freq =  30.0  | cv freq = 

C:\Users\kaixi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


overall pearson correlation (r, p): -0.097 0.653
[eve]
overall pearson correlation (r, p): 0.123 0.566
-- diff: -0.22, 0.086

	-- 3 SEC Timeframe --
cv length 16 eve length 16
[cv]
overall pearson correlation (r, p): -0.1 0.714
[eve]
overall pearson correlation (r, p): 0.135 0.617
-- diff: -0.235, 0.097

	-- 4 SEC Timeframe --
cv length 12 eve length 12
[cv]
overall pearson correlation (r, p): -0.071 0.825
[eve]
overall pearson correlation (r, p): 0.139 0.666
-- diff: -0.211, 0.159

	-- 5 SEC Timeframe --
cv length 9 eve length 9
[cv]
overall pearson correlation (r, p): -0.31 0.417
[eve]
overall pearson correlation (r, p): 0.1 0.799
-- diff: -0.41, -0.382

	-- 6 SEC Timeframe --
cv length 8 eve length 8
[cv]
overall pearson correlation (r, p): -0.174 0.68
[eve]
overall pearson correlation (r, p): 0.207 0.623
-- diff: -0.381, 0.057

** after clean cv data, remove ratio 0.13364055299539168 

eve freq =  30.0  | cv freq =  12.0

**step_32
	-- 1 SEC Timeframe --
cv length 29 eve length 29


In [43]:
cv_38, s30_cv38, s31_cv38, s32_cv38 = input_data('train38')
run('30', s30_cv38, 'train38')
run('31', s31_cv38, 'train38')
run('32', s32_cv38, 'train38')

C:\Users\kaixi\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


** after clean cv data, remove ratio 0.15552855407047383 

eve freq =  30.0  | cv freq =  21.0

**step_30
	-- 1 SEC Timeframe --
cv length 29 eve length 29
[cv]
overall pearson correlation (r, p): -0.255 0.183
[eve]
overall pearson correlation (r, p): -0.028 0.885
-- diff: -0.226, -0.702

	-- 2 SEC Timeframe --
cv length 14 eve length 14
[cv]
overall pearson correlation (r, p): -0.228 0.433
[eve]
overall pearson correlation (r, p): 0.0 1.0
-- diff: -0.228, -0.567

	-- 3 SEC Timeframe --
cv length 9 eve length 9
[cv]
overall pearson correlation (r, p): -0.151 0.699
[eve]
overall pearson correlation (r, p): -0.009 0.981
-- diff: -0.141, -0.282

	-- 4 SEC Timeframe --
cv length 7 eve length 7
[cv]
overall pearson correlation (r, p): -0.239 0.606
[eve]
overall pearson correlation (r, p): -0.051 0.913
-- diff: -0.188, -0.308

	-- 5 SEC Timeframe --
cv length 5 eve length 5
[cv]
overall pearson correlation (r, p): -0.308 0.614
[eve]
overall pearson correlation (r, p): -0.297 0.628
-- diff: -

### train_01 failed (not validated, high scores)

In [44]:
cv_1, s30_cv1, s31_cv1, s32_cv1 = input_data('train01')
run('30', s30_cv1, 'train01')
run('31', s31_cv1, 'train01')
run('32', s32_cv1, 'train01')

C:\Users\kaixi\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


** after clean cv data, remove ratio 0.09591836734693882 

eve freq =  30.0  | cv freq =  15.0

**step_30
	-- 1 SEC Timeframe --
cv length 29 eve length 29
[cv]
overall pearson correlation (r, p): -0.42 0.023
[eve]
overall pearson correlation (r, p): -0.352 0.061
-- diff: -0.068, -0.038

	-- 2 SEC Timeframe --
cv length 14 eve length 14
[cv]
overall pearson correlation (r, p): -0.467 0.092
[eve]
overall pearson correlation (r, p): -0.61 0.02
-- diff: 0.144, 0.072

	-- 3 SEC Timeframe --
cv length 9 eve length 9
[cv]
overall pearson correlation (r, p): -0.575 0.105
[eve]
overall pearson correlation (r, p): -0.534 0.139
-- diff: -0.041, -0.033

	-- 4 SEC Timeframe --
cv length 7 eve length 7
[cv]
overall pearson correlation (r, p): -0.647 0.117
[eve]
overall pearson correlation (r, p): -0.651 0.113
-- diff: 0.004, 0.003

	-- 5 SEC Timeframe --
cv length 5 eve length 5
[cv]
overall pearson correlation (r, p): -0.815 0.093
[eve]
overall pearson correlation (r, p): -0.703 0.186
-- diff: -0.

C:\Users\kaixi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


cv length 9 eve length 9
[cv]
overall pearson correlation (r, p): 0.648 0.059
[eve]
overall pearson correlation (r, p): -0.175 0.653
-- diff: 0.822, -0.594

	-- 4 SEC Timeframe --
cv length 7 eve length 7
[cv]
overall pearson correlation (r, p): 0.569 0.183
[eve]
overall pearson correlation (r, p): -0.21 0.652
-- diff: 0.778, -0.469

	-- 5 SEC Timeframe --
cv length 5 eve length 5
[cv]
overall pearson correlation (r, p): 0.711 0.178
[eve]
overall pearson correlation (r, p): -0.19 0.76
-- diff: 0.901, -0.581

	-- 6 SEC Timeframe --
cv length 4 eve length 4
[cv]
overall pearson correlation (r, p): 0.952 0.048
[eve]
overall pearson correlation (r, p): -0.264 0.736
-- diff: 1.217, -0.688



In [45]:
cv_2, s30_cv2, s31_cv2, s32_cv2  = input_data('train02')
run('30', s30_cv2, 'train02')
run('31', s31_cv2, 'train02')
run('32', s32_cv2, 'train02')

C:\Users\kaixi\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


** after clean cv data, remove ratio 0.04347826086956519 

eve freq =  30.0  | cv freq =  6.0

**step_30
	-- 1 SEC Timeframe --
cv length 29 eve length 29
[cv]
overall pearson correlation (r, p): 0.323 0.1
[eve]
overall pearson correlation (r, p): -0.704 0.0
-- diff: 1.028, 0.1

	-- 2 SEC Timeframe --
cv length 14 eve length 14
[cv]
overall pearson correlation (r, p): 0.445 0.111
[eve]
overall pearson correlation (r, p): -0.733 0.003
-- diff: 1.178, 0.108

	-- 3 SEC Timeframe --
cv length 9 eve length

C:\Users\kaixi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


 9
[cv]
overall pearson correlation (r, p): 0.265 0.491
[eve]
overall pearson correlation (r, p): -0.744 0.021
-- diff: 1.009, 0.469

	-- 4 SEC Timeframe --
cv length 7 eve length 7
[cv]
overall pearson correlation (r, p): 0.465 0.294
[eve]
overall pearson correlation (r, p): -0.77 0.043
-- diff: 1.235, 0.251

	-- 5 SEC Timeframe --
cv length 5 eve length 5
[cv]
overall pearson correlation (r, p): 0.277 0.652
[eve]
overall pearson correlation (r, p): -0.847 0.07
-- diff: 1.124, 0.581

	-- 6 SEC Timeframe --
cv length 4 eve length 4
[cv]
overall pearson correlation (r, p): 0.188 0.812
[eve]
overall pearson correlation (r, p): -0.812 0.188
-- diff: 1.0, 0.625

** after clean cv data, remove ratio 0.1670886075949367 

eve freq =  30.0  | cv freq =  12.0

**step_31
	-- 1 SEC Timeframe --
cv length 29 eve length 29
[cv]
overall pearson correlation (r, p): -0.578 0.002
[eve]
overall pearson correlation (r, p): -0.414 0.032
-- diff: -0.165, -0.03

	-- 2 SEC Timeframe --
cv length 14 eve lengt